In [71]:
import pandas as pd
import scikit_posthocs as sp
from scipy.stats import kruskal, wilcoxon, ttest_ind


df = pd.read_excel("data5.xlsx", sheet_name=1, index_col='state_name', na_values='..')

# оставляем страны, по которым есть полные данные с 2003 по 2022
df = df.groupby('state_name').filter(lambda x: x['year'].min() <= 2002 and x['year'].max() >= 2022)

# удаляем чрезмерно экономически развитые страны (выбросы)
df = df.drop(['Bahrain', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'Trinidad and Tobago', 
              'United Arab Emirates', 'Chile', 'Croatia', 'Hungary', 'Poland', 'Uruguay', 'Equatorial Guinea'])

# удаляем Океанию
df = df[df['region'] != 7]

# выносим названия некоторых колонок в отдельный список для последующих манипуляций
cols = ['libdem_norm', 'property_rights', 'tax_burden', 'government_spending', 
        'business_freedom', 'monetary_freedom', 'trade_freedom', 'investment_freedom', 
        'financial_freedom', 'gov_effectiveness', 'corruption', 'pstab']

cols_1 = ['libdem_norm_diff', 'property_rights_diff', 'tax_burden_diff', 'government_spending_diff', 
        'business_freedom_diff', 'monetary_freedom_diff', 'trade_freedom_diff', 'investment_freedom_diff', 
        'financial_freedom_diff', 'gov_effectiveness_diff', 'corruption_diff', 'pstab_diff']

In [72]:
# разделяем на группы
df_11_plus = df[df.groupby('state_name')['imf_pr'].transform('sum') >= 11]
df_5_to_10 = df[(df.groupby('state_name')['imf_pr'].transform('sum') >= 5) & (df.groupby('state_name')['imf_pr'].transform('sum') <= 10)]
df_4_minus = df[df.groupby('state_name')['imf_pr'].transform('sum') <= 4]

# оставляем только 2002
df_11_plus_2002 = df_11_plus[df_11_plus['year'] == 2002]
df_5_to_10_2002 = df_5_to_10[df_5_to_10['year'] == 2002]
df_4_minus_2002 = df_4_minus[df_4_minus['year'] == 2002]

# оставляем только 2022
df_11_plus_2022 = df_11_plus[df_11_plus['year'] == 2022]
df_5_to_10_2022 = df_5_to_10[df_5_to_10['year'] == 2022]
df_4_minus_2022 = df_4_minus[df_4_minus['year'] == 2022]

In [73]:
# выводим средние значения для 2002 и 2022 в описательных целях
for col in cols:
    print(col)
    print(f'Среднее для группы 4-: {df_4_minus_2002[col].mean().round(2)} (2002); {df_4_minus_2022[col].mean().round(2)} (2022), дельта {(df_4_minus_2022[col].mean().round(2) - df_4_minus_2002[col].mean().round(2)).round(2)}')
    print(f'Среднее для группы 5-10: {df_5_to_10_2002[col].mean().round(2)} (2002); {df_5_to_10_2022[col].mean().round(2)} (2022), дельта {(df_5_to_10_2022[col].mean().round(2) - df_5_to_10_2002[col].mean().round(2)).round(2)}')
    print(f'Среднее для группы 11+: {df_11_plus_2002[col].mean().round(2)} (2002); {df_11_plus_2022[col].mean().round(2)} (2022), дельта {(df_11_plus_2022[col].mean().round(2) - df_11_plus_2002[col].mean().round(2)).round(2)}')
    print()
    # print(col)
    # print(f'Δ{(df_4_minus_2022[col].mean().round(2) - df_4_minus_2002[col].mean().round(2)).round(2)} ({df_4_minus_2002[col].mean().round(2)}, {df_4_minus_2022[col].mean().round(2)})')
    # print(f'Δ{(df_5_to_10_2022[col].mean().round(2) - df_5_to_10_2002[col].mean().round(2)).round(2)} ({df_5_to_10_2002[col].mean().round(2)}, {df_5_to_10_2022[col].mean().round(2)})')
    # print(f'Δ{(df_11_plus_2022[col].mean().round(2) - df_11_plus_2002[col].mean().round(2)).round(2)} ({df_11_plus_2002[col].mean().round(2)}, {df_11_plus_2022[col].mean().round(2)})')
    # print()

libdem_norm
Среднее для группы 4-: 40.46 (2002); 32.87 (2022), дельта -7.59
Среднее для группы 5-10: 43.31 (2002); 40.42 (2022), дельта -2.89
Среднее для группы 11+: 32.48 (2002); 36.43 (2022), дельта 3.95

property_rights
Среднее для группы 4-: 37.41 (2002); 45.73 (2022), дельта 8.32
Среднее для группы 5-10: 38.67 (2002); 45.22 (2022), дельта 6.55
Среднее для группы 11+: 38.22 (2002); 42.23 (2022), дельта 4.01

tax_burden
Среднее для группы 4-: 72.37 (2002); 81.1 (2022), дельта 8.73
Среднее для группы 5-10: 73.95 (2002); 80.41 (2022), дельта 6.46
Среднее для группы 11+: 69.45 (2002); 80.61 (2022), дельта 11.16

government_spending
Среднее для группы 4-: 77.42 (2002); 75.96 (2022), дельта -1.46
Среднее для группы 5-10: 75.57 (2002); 73.25 (2022), дельта -2.32
Среднее для группы 11+: 77.14 (2002); 80.3 (2022), дельта 3.16

business_freedom
Среднее для группы 4-: 57.78 (2002); 59.1 (2022), дельта 1.32
Среднее для группы 5-10: 57.5 (2002); 57.85 (2022), дельта 0.35
Среднее для группы 11+:

In [74]:
# выявляем возможные различия в развитии институтов для трёх рассматриваемых групп в 2002 г.
for col in cols:
    data = [df_11_plus_2002[col], df_5_to_10_2002[col], df_4_minus_2002[col]]
    stat, p_value = kruskal(*data)
    print(col)
    print(f"Kruskal-Wallis H-statistic: {stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if p_value < 0.05:
        print()
        print("Есть статистически значимые различия, выполняем тест Данна:")
        print(sp.posthoc_dunn(data, p_adjust='holm'))
    else:
        print('Нет различий между группами')
    print()

libdem_norm
Kruskal-Wallis H-statistic: 2.9767
P-value: 0.2258
Нет различий между группами

property_rights
Kruskal-Wallis H-statistic: 0.2588
P-value: 0.8786
Нет различий между группами

tax_burden
Kruskal-Wallis H-statistic: 1.9520
P-value: 0.3768
Нет различий между группами

government_spending
Kruskal-Wallis H-statistic: 0.1639
P-value: 0.9213
Нет различий между группами

business_freedom
Kruskal-Wallis H-statistic: 0.8284
P-value: 0.6609
Нет различий между группами

monetary_freedom
Kruskal-Wallis H-statistic: 1.8896
P-value: 0.3888
Нет различий между группами

trade_freedom
Kruskal-Wallis H-statistic: 0.1379
P-value: 0.9334
Нет различий между группами

investment_freedom
Kruskal-Wallis H-statistic: 5.1502
P-value: 0.0761
Нет различий между группами

financial_freedom
Kruskal-Wallis H-statistic: 2.9662
P-value: 0.2269
Нет различий между группами

gov_effectiveness
Kruskal-Wallis H-statistic: 5.8789
P-value: 0.0529
Нет различий между группами

corruption
Kruskal-Wallis H-statistic:

In [75]:
# выявляем возможные различия в развитии институтов для трёх рассматриваемых групп в 2022 г.
for col in cols:
    data = [df_11_plus_2022[col], df_5_to_10_2022[col], df_4_minus_2022[col]]
    stat, p_value = kruskal(*data)
    print(col)
    print(f"Kruskal-Wallis H-statistic: {stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if p_value < 0.05:
        print()
        print("Есть статистически значимые различия, выполняем тест Данна:")
        print(sp.posthoc_dunn(data, p_adjust='holm'))
    else:
        print('Нет различий между группами')
    print()

libdem_norm
Kruskal-Wallis H-statistic: 2.0856
P-value: 0.3525
Нет различий между группами

property_rights
Kruskal-Wallis H-statistic: 0.9398
P-value: 0.6251
Нет различий между группами

tax_burden
Kruskal-Wallis H-statistic: 0.4555
P-value: 0.7963
Нет различий между группами

government_spending
Kruskal-Wallis H-statistic: 4.1926
P-value: 0.1229
Нет различий между группами

business_freedom
Kruskal-Wallis H-statistic: 6.7696
P-value: 0.0339

Есть статистически значимые различия, выполняем тест Данна:
          1         2         3
1  1.000000  0.175579  0.032682
2  0.175579  1.000000  0.376458
3  0.032682  0.376458  1.000000

monetary_freedom
Kruskal-Wallis H-statistic: 1.3367
P-value: 0.5126
Нет различий между группами

trade_freedom
Kruskal-Wallis H-statistic: 4.1634
P-value: 0.1247
Нет различий между группами

investment_freedom
Kruskal-Wallis H-statistic: 2.5335
P-value: 0.2817
Нет различий между группами

financial_freedom
Kruskal-Wallis H-statistic: 0.1377
P-value: 0.9335
Нет 

In [76]:
# создаём датафреймы для сравнения динамики
df_11_plus_ = df_11_plus[df_11_plus['year'] > 2002]
df_5_to_10_ = df_5_to_10[df_5_to_10['year'] > 2002]
df_4_minus_ = df_4_minus[df_4_minus['year'] > 2002]

In [77]:
# сравниваем средние изменения институциональных переменных у трёх рассматриваемых групп стран за период 2003-2022
for col in cols_1:
    data = [df_11_plus_[col], df_5_to_10_[col], df_4_minus_[col]]
    stat, p_value = kruskal(*data)
    print(col)
    print(f"Kruskal-Wallis H-statistic: {stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if p_value < 0.05:
        print()
        print("Есть статистически значимые различия, выполняем тест Данна:")
        print(sp.posthoc_dunn(data, p_adjust='holm'))
    else:
        print('Нет различий между группами')
    print()

libdem_norm_diff
Kruskal-Wallis H-statistic: 5.8151
P-value: 0.0546
Нет различий между группами

property_rights_diff
Kruskal-Wallis H-statistic: 3.2914
P-value: 0.1929
Нет различий между группами

tax_burden_diff
Kruskal-Wallis H-statistic: 9.2984
P-value: 0.0096

Есть статистически значимые различия, выполняем тест Данна:
          1         2         3
1  1.000000  0.024033  0.773165
2  0.024033  1.000000  0.018834
3  0.773165  0.018834  1.000000

government_spending_diff
Kruskal-Wallis H-statistic: 0.6773
P-value: 0.7127
Нет различий между группами

business_freedom_diff
Kruskal-Wallis H-statistic: 3.1161
P-value: 0.2105
Нет различий между группами

monetary_freedom_diff
Kruskal-Wallis H-statistic: 0.1860
P-value: 0.9112
Нет различий между группами

trade_freedom_diff
Kruskal-Wallis H-statistic: 0.0613
P-value: 0.9698
Нет различий между группами

investment_freedom_diff
Kruskal-Wallis H-statistic: 1.2623
P-value: 0.5320
Нет различий между группами

financial_freedom_diff
Kruskal-Wa

In [78]:
for col in cols_1:
    print(col)
    print(f'Среднее для группы 11+: {df_11_plus_[col].mean().round(2)}')
    print(f'Среднее для группы 5-10: {df_5_to_10_[col].mean().round(2)}')
    print(f'Среднее для группы 4-: {df_4_minus_[col].mean().round(2)}')
    print()

libdem_norm_diff
Среднее для группы 11+: 0.2
Среднее для группы 5-10: -0.14
Среднее для группы 4-: -0.38

property_rights_diff
Среднее для группы 11+: 0.2
Среднее для группы 5-10: 0.33
Среднее для группы 4-: 0.42

tax_burden_diff
Среднее для группы 11+: 0.56
Среднее для группы 5-10: 0.32
Среднее для группы 4-: 0.44

government_spending_diff
Среднее для группы 11+: 0.16
Среднее для группы 5-10: -0.12
Среднее для группы 4-: -0.07

business_freedom_diff
Среднее для группы 11+: -0.26
Среднее для группы 5-10: 0.02
Среднее для группы 4-: 0.07

monetary_freedom_diff
Среднее для группы 11+: -0.0
Среднее для группы 5-10: -0.03
Среднее для группы 4-: -0.01

trade_freedom_diff
Среднее для группы 11+: 0.29
Среднее для группы 5-10: 0.31
Среднее для группы 4-: 0.36

investment_freedom_diff
Среднее для группы 11+: 0.24
Среднее для группы 5-10: -0.13
Среднее для группы 4-: 0.11

financial_freedom_diff
Среднее для группы 11+: 0.03
Среднее для группы 5-10: -0.08
Среднее для группы 4-: 0.17

gov_effectiv

In [79]:
for col in cols:
    u_stat, p_value = wilcoxon(df_4_minus_2002[col], df_4_minus_2022[col], alternative='two-sided')
    print(col)
    print(f"W-statistic: {u_stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    t_stat, p_value = ttest_ind(df_4_minus_2002[col], df_4_minus_2022[col], alternative='two-sided')
    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_value:.4f}\n")

libdem_norm
W-statistic: 76.0000
P-value: 0.0055
T-statistic: 1.0259
P-value: 0.3097

property_rights
W-statistic: 82.5000
P-value: 0.0105
T-statistic: -1.6333
P-value: 0.1084

tax_burden
W-statistic: 45.0000
P-value: 0.0005
T-statistic: -3.0972
P-value: 0.0031

government_spending
W-statistic: 144.5000
P-value: 0.2850
T-statistic: 0.3945
P-value: 0.6949

business_freedom
W-statistic: 171.0000
P-value: 0.6654
T-statistic: -0.4273
P-value: 0.6710

monetary_freedom
W-statistic: 170.0000
P-value: 0.6617
T-statistic: 0.0434
P-value: 0.9656

trade_freedom
W-statistic: 77.5000
P-value: 0.0074
T-statistic: -1.8641
P-value: 0.0680

investment_freedom
W-statistic: 113.5000
P-value: 0.4550
T-statistic: -0.4021
P-value: 0.6892

financial_freedom
W-statistic: 59.0000
P-value: 0.2415
T-statistic: -0.6163
P-value: 0.5404

gov_effectiveness
W-statistic: 180.0000
P-value: 0.8408
T-statistic: -0.3940
P-value: 0.6952

corruption
W-statistic: 147.0000
P-value: 0.3242
T-statistic: 0.4457
P-value: 0.6577



In [80]:
for col in cols:
    u_stat, p_value = wilcoxon(df_5_to_10_2002[col], df_5_to_10_2022[col], alternative='two-sided')
    print(col)
    print(f"W-statistic: {u_stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    t_stat, p_value = ttest_ind(df_5_to_10_2002[col], df_5_to_10_2022[col], alternative='two-sided')
    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_value:.4f}\n")

libdem_norm
W-statistic: 186.0000
P-value: 0.3388
T-statistic: 0.5056
P-value: 0.6151

property_rights
W-statistic: 112.0000
P-value: 0.0132
T-statistic: -2.1471
P-value: 0.0360

tax_burden
W-statistic: 102.0000
P-value: 0.0073
T-statistic: -2.7783
P-value: 0.0074

government_spending
W-statistic: 170.0000
P-value: 0.2054
T-statistic: 0.4748
P-value: 0.6367

business_freedom
W-statistic: 226.0000
P-value: 0.8936
T-statistic: -0.1440
P-value: 0.8860

monetary_freedom
W-statistic: 209.0000
P-value: 0.6288
T-statistic: 0.2199
P-value: 0.8267

trade_freedom
W-statistic: 92.0000
P-value: 0.0039
T-statistic: -2.6821
P-value: 0.0095

investment_freedom
W-statistic: 178.0000
P-value: 0.7905
T-statistic: 0.5950
P-value: 0.5542

financial_freedom
W-statistic: 128.0000
P-value: 0.5134
T-statistic: 0.4349
P-value: 0.6652

gov_effectiveness
W-statistic: 203.0000
P-value: 0.5561
T-statistic: 0.3479
P-value: 0.7292

corruption
W-statistic: 191.0000
P-value: 0.4045
T-statistic: 0.4183
P-value: 0.6773


In [81]:
for col in cols:
    u_stat, p_value = wilcoxon(df_11_plus_2002[col], df_11_plus_2022[col], alternative='two-sided')
    print(col)
    print(f"W-statistic: {u_stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    t_stat, p_value = ttest_ind(df_11_plus_2002[col], df_11_plus_2022[col], alternative='two-sided')
    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_value:.4f}\n")

libdem_norm
W-statistic: 170.0000
P-value: 0.2054
T-statistic: -0.8770
P-value: 0.3841

property_rights
W-statistic: 170.0000
P-value: 0.1986
T-statistic: -1.1405
P-value: 0.2588

tax_burden
W-statistic: 37.0000
P-value: 0.0002
T-statistic: -3.8518
P-value: 0.0003

government_spending
W-statistic: 125.0000
P-value: 0.0455
T-statistic: -0.7913
P-value: 0.4320

business_freedom
W-statistic: 148.0000
P-value: 0.0822
T-statistic: 1.8215
P-value: 0.0737

monetary_freedom
W-statistic: 212.5000
P-value: 0.6808
T-statistic: 0.0311
P-value: 0.9753

trade_freedom
W-statistic: 101.0000
P-value: 0.0118
T-statistic: -1.8175
P-value: 0.0743

investment_freedom
W-statistic: 95.0000
P-value: 0.1145
T-statistic: -1.4010
P-value: 0.1665

financial_freedom
W-statistic: 75.0000
P-value: 0.9415
T-statistic: -0.2246
P-value: 0.8231

gov_effectiveness
W-statistic: 211.0000
P-value: 0.6702
T-statistic: 0.1499
P-value: 0.8814

corruption
W-statistic: 175.0000
P-value: 0.2449
T-statistic: -0.8352
P-value: 0.407